# CA 1

## a)

Let us define $L=\frac{1}{N}\sum\limits_{i\in[N]} \lvert\lvert w^Tx_i - y_i\rvert\rvert^2 + \lambda\lvert\lvert w\rvert\rvert^2$

Then 
\begin{align}
\frac{\partial L}{\partial w}&=\frac{1}{N} \sum\limits_{i\in[N]} \frac{\partial x_i^T w w^T x_i}{\partial w} - 2\frac{\partial y_i^T w^T x_i}{\partial w} + 2\lambda w\\
&=\frac{1}{N} \sum\limits_{i\in[N]} 2 x_i x_i^T w - 2x_i y_i^T + 2\lambda w\\
&=0,
\end{align}
so
\begin{align}
\frac{1}{N} \sum\limits_{i\in[N]} x_i y_i^T =  \frac{1}{N}\sum\limits_{i\in[N]}  x_i x_i^T w  + \lambda w
\end{align}


So $$w^*=(XX^T + \lambda I)^{-1} X Y^T$$
where $X=[x_1,x_2,\dots,x_N]$ and $Y=[y_1,y_2,\dots,y_N]$

In [2]:
##imports from libraries
import pandas as pd
import numpy as np
import time
from sklearn import linear_model

In [9]:
## Load data and preprocessing

## Example of reading dataset ''crimedata.csv'' : 
data = pd.read_csv("household_power_consumption.txt", sep=";")
Y = data.iloc[:,2:4]
X = data.iloc[:,4:8]
## You can observe the shape of data by 
print(X.shape)
print(Y.shape)

X_ = np.array(X.replace({'?':0})).T
Y_ = np.array(Y.replace({'?':0})).T

X_ = X_.astype(float)
Y_ = Y_.astype(float)

#Y_ = np.array(Y).reshape(Y.shape[1],Y.shape[0])

(2075259, 4)
(2075259, 2)
[[234.84 233.63 233.29 ... 239.82 239.7  239.55]
 [ 18.4   23.    23.   ...   3.8    3.8    3.8 ]
 [  0.     0.     0.   ...   0.     0.     0.  ]
 [  1.     1.     2.   ...   0.     0.     0.  ]]
[[4.216 5.36  5.374 ... 0.938 0.934 0.932]
 [0.418 0.436 0.498 ... 0.    0.    0.   ]]


In [32]:
## Closed form solution and optimal linear regressor

# Define lambda here:
lam = 0 # change the value

start = time.time()
## Calculate the closed-form solution here:
A=X_.dot(X_.T)+lam*np.eye(X_.shape[0])
w=np.linalg.inv(A).dot(X_).dot(Y_.T)
end = time.time()
print('my result')
print('time=',end-start,' seconds')
print('w=\n',w)
print('MSE',np.linalg.norm(Y_-w.T.dot(X_),ord='fro')/X_.shape[1],'\n\n')


reg = linear_model.Ridge(alpha=0)
start = time.time()
## Find the optimal linear regressor here:
reg.fit(X_.T,Y_.T)
end = time.time()
w_sk=reg.coef_.T
print('sklearn:')
print('time=',end-start,' seconds')
print('w=\n',w_sk)
print('MSE',np.linalg.norm(Y_-w_sk.T.dot(X_),ord='fro')/X_.shape[1],'\n\n')

my result
time= 0.10849595069885254  seconds
w=
 [[-5.06370205e-05  3.83915746e-04]
 [ 2.39266696e-01  6.62357767e-03]
 [-1.26168605e-03 -7.18129903e-05]
 [-1.44871660e-03  5.09803289e-04]]
MSE 8.221321832090663e-05 


sklearn:
time= 0.12466740608215332  seconds
w=
 [[ 1.33213175e-06  3.77957792e-04]
 [ 2.39283423e-01  6.62165997e-03]
 [-1.26214423e-03 -7.17604620e-05]
 [-1.44939987e-03  5.09881622e-04]]
MSE 8.267688332411155e-05 


